# Планирование экспериментов

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#&quot;Проклятие-размерности&quot;-и-как-его-избежать" data-toc-modified-id="&quot;Проклятие-размерности&quot;-и-как-его-избежать-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>"Проклятие размерности" и как его избежать</a></span></li><li><span><a href="#Физические-и-численные-эксперименты" data-toc-modified-id="Физические-и-численные-эксперименты-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Физические и численные эксперименты</a></span></li><li><span><a href="#Проектирование-предварительных-экспериментов-(скрининг)" data-toc-modified-id="Проектирование-предварительных-экспериментов-(скрининг)-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Проектирование предварительных экспериментов (скрининг)</a></span><ul class="toc-item"><li><span><a href="#Оценка-распределения-элементарных-эффектов" data-toc-modified-id="Оценка-распределения-элементарных-эффектов-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Оценка распределения элементарных эффектов</a></span></li></ul></li></ul></div>

In [1]:
import numpy as np
import matplotlib.pyplot as plt

Задача инженерного дизайна: построить "суррогатную" модель _f_, которая легко оценивается и имитирует сложное поведение неизвестного объекта __f__

Обозначим $D$ (подмножество k-мерного R-пространства, $D \subseteq R^k$) - так называемую ___область проектирования___, в которой функция f(x) непрерывна. Но мы можем получать только дискретные значения функции f {$x_i$ → $y_i$ = f($x_i$)|i = 1..n}. Это довольно дорогостоящая процедура. Так что наша задача состоит в том, чтобы используя некоторый набор этих дискретных данных (__выборку__), сконструировать приближённую функцию _f_, с помощью которой можно легко (с малыми затратами по времени и памяти) производить предсказания значения функции f для любого __x__ их $D$.

$D$ - область проектирования  
__x__ - проектные переменные  
__f__ - целевая функция  
_f_   - суррогатная модель.  


Большая часть этой книги - способы конструирования _f_ по заданной выборке. За исключением нескольких случаев, для которых уже найдены точные математические решения...  
Некоторым моделям необходимо минимальное число элементов в выборке _n_...

Важно сказать, что сконструированные таким образом функции _f_ по заданной выборке не всегда хорошо обобщаются на всю область $D$. Всё зависит от пришедшей на вход выборки.  
Получение этой выборки не всегда зависит от нас: её могли получить исследуя другие вопросы. Однако далее будет описано, как в случае возможности создания своей собственной выборки сделать такую выборку, которая позволит создать хорошо обобщающую данные суррогатную модель _f_.

## "Проклятие размерности" и как его избежать

Очевидно, чем больше в задаче переменных, тем сложнее будет функция измерения, с помощью которой мы хотим достаточно точно определить местоположение объекта. Заметим, что если некоторый уровень точности определения положения в одномерном пространстве достигается взятием выборки из _n_ позиций, то в _k_-мерном пространстве для достижения такой же точности необходимо взять выборку из _n$^k$_ точек. Это и есть "проклятие размерности". Поэтому важно с самого начала свести количество _проектных переменных k_ к минимуму.  
Задача состоит в том, чтобы среди всех переменных выделить существенные. Но сначала нам нужно сделать несколько общих замечаний о физических и вычислительных экспериментах, двух источниках, которые могут быть использованы для получения целевой функции

## Физические и численные эксперименты

Существуют 3 источника ошибок, из-за которых результаты эксперимента отклоняются от теоретических расчётов:

* __человеческая ошибка__ - осуществляется самим экспериментатором;  
* __систематическая ошибка__ - появляется из-за недостаточного соответствия между теорией и экспериментом;
* __случайная ошибка__ - появляется из-за наличия погрешности измерений.

Главным отличием между последними двумя пунктами является повторяемость. Систематическая составляющая вносит в экспериментальную ошибку всегда одно и то же значение, а случайная - каждый раз будет отличаться, и при большом числе экспериментов она будет принимать как положительные, так и отрицательные значения.  
Численные эксперименты (эксперименты, смоделированные на ЭВМ) также подвержены экспериментальной ошибке, возникающей в результате:

* __человеческая ошибка__ - 'баги' и ошибки в написанном коде, неверное введённые граничные условия при решении ДУЧП (дифференциального уравнения в частных производных) и т.д.;
* __систематическая ошибка__ - например, математическая модель невязкого обтекания тела (аппроксимация, иногда используемая для экономии вычислительного времени) будет недооценивать силы сопротивления, действующие на тело (потому что в течении на самом деле присутствует вязкость). Другой пример - ошибка, вызванная конечностью разрешения численного моделирования (например, ошибка, вызванная малым числом узлов расчётной сетки). Хоть этот тип ошибки может как переоценить значение некоторой величины, так и недооценить, но при повторении эксперимента он сделает точно также.  

Таким образом, разница между физическими и численными экспериментами заключается в том, что на вторые не влияет случайная ошибка - численные эксперименты ___детерминированы___  

__Небольшое замечание:__  
> Физические экспериментаторы часто используют слово *шум*, имея в виду случайную ошибку. Однако в некоторой литературе по численным экспериментам также встречается слово *шум*, которое должно относится к систематическим ошибкам, потому что там нет случайных.  
Чтобы не запутаться, в дальнейшем, когда мы будем говорить об ошибках физических экспериментов, будем писать слово *шум* без кавычек,а когда об ошибках численных экспериментов - будем писать "шум" - в кавычках
>  



## Проектирование предварительных экспериментов (скрининг)

Ранее мы увидели, насколько важно минимизировать число проектных переменных $x_1, x_2, ..., x_k$ перед тем, как мы попытаемся смоделировать целевую функцию __f__. Но как нам добиться такого ___скрининга___ (так мы будем называть этот процесс в дальнейшем), чтобы наш анализ всё ещё имел смысл?  
Дифференцируемость функции __f__ в области проектирования $D$ по каждой переменной: $$\frac{\partial f}{\partial x_i}|_x$$ является важным критерием для классификации:
* если $\frac{\partial f}{\partial x_i}|_x = 0, \forall x \in D$, то переменной x_i можно смело пренебречь,
* если $\frac{\partial f}{\partial x_i}|_x = const \neq 0, \forall x \in D$, то влияние переменной $x_i$ линейно и аддитивно,
* если $\frac{\partial f}{\partial x_i}|_x = g(x_i) \neq const, \forall x \in D$, то $f$ нелинейна по $x_i$,
*  если $\frac{\partial f}{\partial x_i}|_x = g(x_i),\forall x \in D$, где  $g(x_i, x_j, ...) \neq const$, то $f$ нелинейна по $x_i$ и участвует во взаимодействиях с $x_j, ...$  

Приведенная выше классификация является всего лишь терминологической формулировкой, т.к. на практике мы не можем измерить $\frac{\partial f}{\partial x_i}|_x$ во всей области проектирования. Скрининговое исследование - трудная и ресурсоёмкая задача. Однако нет жёсткого правила относительно того, сколько времени должно быть потрачено на скринниг переменных, т.к. это в значительной степени зависит от задачи. Если мы ожидаем, что многими переменными можно будет пренебречь, то тщательное скрининговое исследование может значительно повысить точность последующей "суррогатной" модели. Однако, если есть (инженерные) основания, полагать, что большинство переменных имеют значительное влияние на целевую функцию, то целесообразно сосредоточить усилия на самом моделировании.  
О методах скрининга входных переменных написано много статей (Jones et al., 1998). Их принципы работы варьируются в зависимости от допущений, которые они делают относительно целевой функции и переменных. Здесь мы остановимся на алгоритме, который в 1991 году описал Моррис. Почему мы выбрали именно этот алгоритм? Потому что единственное допущение, которое он делает, заключается в том, что целевая функция детерминирована (это общая особенность для большинства вычислительных моделей).

### Оценка распределения элементарных эффектов

Без ограничения общности предположим, что наша область проектирования $D$ является $k$-мерным единичным кубом, т.е. $D=[0,1]^k$ Это предположение упростит дальнейшие рассуждения.  

Прежде чем приступить к описанию алгоритма Морриса, нам необходимо определить важное статистическое понятие. Ограничим нашу область проектирования $D$ $k$-мерной $p$-уровневой факториальной сеткой, т.е. $x_i \in \{0, \frac{1}{p-1}, \frac{2}{p-1}, ... , 1\}$ для   $i = 1,...,k$. Для зараннее заданного значения  __x__ $\in D$ определим ___элементарное воздействие___ от  $x_i$ $d_i(x)$, равное 
$$d_i(x)=\frac{y(x_1,x_2,...,x_{i-1},x_i+\Delta, x_{i+1},...,x_k)-y(x)}{\Delta},$$  
где $\Delta = \frac{\xi}{p-1}, \xi \in \mathbb{N^*}$ и $x \in D$ такой, что его компоненты $x_i \leq 1-\Delta$

Метод Мориса используется для оценки параметров распределения элементарных воздействий, связанных с каждой переменной. Принцип заключается в том, что большая величина _центральной тенденции_ (мат. ожидания???) указывает на переменную, имеющую сильное влияние на целевую функцию в области проектирования, а большая _мера разброса_ (дисперсии???) указывает на переменную, по которой __f__ не линейна и участвует во взаимодействиях с другими переменными. На практике мы оцениваем выборочное среднее(среднее арифметическое??? $\overline{x} = \frac{1}{n} \sum\limits_{i=1}^n x_i$) и выборочное стандартное отклонение(Среднеквадратичное отклонение ??? $\sigma = \sqrt{D[x]}$) набора значений $d_i(x)$, рассчитанных в различных частях проектного пространства.

Ясно, что желательно сгенерировать предварительный план выборки __X__ так, чтобы каждая оценка целевой функции __f__ участвовала в вычислении двух элементарных воздействий (вместо одного, если бы использовали случайное распределение __x__ и затем добавляли $\Delta$ к одной из его компонент). Также, план выборки должен дать нам вполне конкретное число (например, _r_) элементарных воздействий для каждой переменной

Пусть __B__ является ___матрицей выборки___, т.е. матрицей размерности $k+1 \times k$ состоящей из нулей и единиц с тем свойством, что для кождого столбца $i=\overline{1,k})$ есть две строки __B__, которые отличаются в своей i-ой позиции (далее мы напишем функцию реализующую такую матрицу). Затем мы вычисляем __B__$^*$ - ___случайную ориентацию___ матрицы __B__:
$$B^* = (1_{k+1,1}x^* + (\frac{\Delta}{2})[(2B - 1_{k+1,k})D^* + 1_{k+1,k}])P^*,$$
где $D^*$ - это $k$-мерная диагональная матрица, где каждый элемент равен +1 или -1 с равной вероятностью, $1_{k+1,k}$ - это матрица единиц, $x^*$ - это случайно выбранная точка нашей дискретизированной $p$-уровневой области проектирования (ограниченной по краям $\Delta$, как было сказано ранее) и $P^* (k \times k)$ - это матрица случайной перестановки столбцов, в которой каждый столбец содердит всего одну единицу, а все остальные элементы равны нулю, при этом у любой пары столбцов единицы стоят в разных строках. Далее приведена реализация данного выражения на языке Python  

In [15]:
def randorient(k,p, xi):
    """
    Генерирует случайную ориентацию скрининговой матрицы.

    Вход:
        k - число проектных перемненных
        p - число дискретных уровней в каждой размерности области проектирования
        xi - коэффициент элементарного воздействия(???)

    Выход:
        Bstar - случайная ориентация матрицы
    """
    
    Delta = xi/(p-1)
    m = k+1
    
    # сужение p-уровневой сетки
    xs = np.hstack((np.arange(0, 1-Delta, 1/(p-1)),1-Delta))
    xsl = len(xs)    
    
    #Матрица выборки
    B = np.append(np.zeros((1,k)), np.tril(np.ones((k,k))))
    
    #Рандомизация
    
    #k-мерная диагональная матрица, где каждый элемент 
    #равен +1 или -1 с равной вероятностью
    Dstar = np.diag(np.int64(np.round(np.random.rand(k)))*2-1)
    
    #Перестановочная матрица
    Pstar = np.zeros((k,k))
    rp = np.random.permutation(k)
    for i in np.arange(k):
        Pstar[i][rp[i]] = 1
    
    #случайно выбранная точка
    xstar = xs[np.int64(np.floor(np.random.rand(k)*xsl)+1)]
    
    Bstar = (np.ones((m,1)) * xstar + (Delta/2)) * ((2*B - np.ones((m,k)))*Dstar + np.ones(m,k)) * Pstar
    return Bstar

 Чтобы получить _r_ элементарных воздействий для каждой переменной  план скрининга строится из _r_ случайных ориентаций:
 $$X = \left[\begin{array}{crl} B^*_1\\ B^*_2\\ ... \\ B^*_r \end{array}\right] $$

In [3]:
def screeningplan(k, p , xi, r):
    """
    Генерирует Моррисовский скрининговый план с заданным 
    числом элементарных воздействий для каждой переменной
    
    Вход:
        k - число проектных перемненных
        p - число дискретных уровней в каждой размерности области проектирования
        xi - коэффициент элементарного воздействия(???)
        r - число случайных ориентаций (=число элементарных 
            воздействий на кажду переменную)
    Выход:
        X - скрининговый план внутри к-мерного куба [0,1]^k
    """
    X = []
    for i in np.arange(r):
        X = np.append(X, randorient(k,p,xi))
    
    return X

Затем мы посчитаем значение функции _f_ для каждой строки $X$. В дальнейшем мы будем хранить эти значения целевой функции в векторе-столбце $t_{r(k+1) \times 1}$. Взяв по одной случайной ориентации за раз, соседние строки из матрицы скринингового плана и соответствующие значения функции f могут быть вставлены в уравнение (2) для получения $k$ элементарных воздействий (по одному для каждой переменной).  
После того как собрана выборка из _r_ элементарных воздейтвий, средние значения выборок и стандартные отклонения выборок могут быть вычислены и представлены на одной и той же диаграмме для сравнения.

In [4]:
def screeningplot(X, Objhandle, Range, xi, p, Labels):
    """
    Генерирует график скрининга переменных элементарных воздействий
    Вход:
        X - скрининговый план внутри к-мерного куба [0,1]^k
        Objhandle - целевая функция
        Range - матрица 2 на к (к - число проектных переменных) нижних(первая строка) 
                и верхних(вторая строка) границ для каждой переменной
        p - число дискретных уровней в каждой размерности области проектирования
        xi - коэффициент элементарного воздействия(???)
        Labels - массив (1 на к) имён переменных
    """
    k = X.shape[1] #число столбцов матрицы X =  число проектных переменных
    r = X.shape[0]/k - 1 #почему нельзя просто написать  к+1???
    t = []
    
    for i in np.arange(X.shape[0]):
        X[i,:] = Range[0,:] + X[i:0]*(Range[1,:] - Range[0,:])
        t.append(list(map(Objhandle, X[i,:])))
    

    F = np.zeros((k,r))
    for i in np.arange(r):
        for j in np.arange(i*(k+1), i*(k+1)+k):
            F[np.where(X[j,:] - X[j+1,:] != 0)][i] = (t[j+1] - t[j])/(xi/(p-1))
    
        
    #вычисляем статистические величины
    ssd=[]
    sm=[]
    for i in np.arange(k):
        ssd.append(np.std(F[i,:]))
        sm.append(np.mean(F[i,:]))
        
    size = 10
    plt.figure(figsize=(size,size))
    plt.xlabel('Sample_means', fontsize=16)
    plt.ylabel('Sample_standart_deviations', fontsize=16)
    plt.xlim(min(sm), max(sm))
    plt.ylim(min(ssd), max(ssd))
    
    for i in np.arange(k):
        plt.text(sm[i],ssd[i],Labels[i], fontsize=25)

Прежде чем проиллюстрировать процесс примером инженерного проектирования, стоит упомянуть два сценария, где развертывание описанного выше алгоритма требует особой осторожности.  
Во-первых, если размерность k пространства относительно мала и мы можем позволить себе большое значение _r_, то следует иметь в виду повышенную вероятность того, что одна и та же конструкция появится дважды в $X$. Если ответы в точках выборки детерминированы, то, конечно, нет смысла повторять оценку. Эта проблема возникает не особенно часто, так как при скрининге пространств с большими размерностями обычно требуется _большое_ количество элементарных воздействий. 
Во-вторых, программы численного моделирования иногда не возвращают разумный результат из-за ошибок сетки, неспособности решения дифференциального уравнения в частных производных сходиться и т. д. С точки зрения скрининга это важно, поскольку все случайные ориентации $B^*$  будут под угрозой, если вычисление целевой функции не удаётся для одной из точек.  

#### Весовая функция крыла от 10 переменных

Рассмотрим следующее аналитическое выражение, используемое в качестве концептуального уровня оценки веса крыла легкого самолета:
$$W = 0.036S^{0.758}_wW^{0.0035}_{fw}(\frac{A}{cos^2\Lambda})^{0.6}q^{0.006}\lambda^{0.04}(\frac{100tc}{cos\Lambda})^{-0.3}(N_zW_{dg})^{0.49}+S_wW_p.$$

Таблица 1.1 содержит номенклатуру символов, используемых в уравнении (5), а также набор основных значений, приблизительно представляющих самолет Cessna C172 Skyhawk и несколько произвольно выбранный диапазон для каждой переменной. Эти основные значения и диапазоны были использованы для создания заполненного контурного графика весовой функции (см. рис. 1.1) путем попарного изменения входных данных и сохранения оставшихся переменных на базовом значении.

__Таблица 1.1.__  Номенклатура примера задачи скрининга с десятью переменными

| Символ    | Параметр                                            | Основное значение | Мин.знач | Макс.знач |
| ------    | ------------------------                            | ----------------- | -------- | --------- |
| $S_w$     | площадь крыла                                       | 174               | 150      | 200       |
| $W_{fw}$  | вес топлива в крыле                                 | 252               | 220      | 300       |
| $A$       | Соотношение сторон                                  | 7.52              | 6        | 10        |
| $\Lambda$ | Четверть-хордовая развертка (град)                  | 0                 | -10      | 10        |
| $q$       | Скоростной напор???                                 | 34                | 16       | 45        |
| $\lambda$ | Коэффициент конусности                              | 0.672             | 0.5      | 1         |
| $tc$      | Отношение толщины аэродинамического профиля к хорде | 0.12              | 0.08     | 0.18      |
| $N_z$     | Предельный коэффициент нагрузки                     | 3.8               | 2.5      | 6         |
| $W_{dg}$  | Полная масса лётной конструкции                     | 2000              | 1700     | 2500      |
| $W_p$     | Вес краски                                          | 0.064             | 0.025    | 0.08      |


[image 1.1]: ///home/ruslan/Изображения/graph

![](///home/ruslan/Изображения/graph/150x150)

![graph](img/graph "график")  

__Рис. 1.1.__ Легкий самолет с массой крыла $W$. Каждая плитка показывает контур весовой функции (уравнение (5)) по сравнению с двумя из десяти переменных, а остальные восемь переменных удерживаются на базовом значении

Итак, что же показывает сюжет с точки зрения активности переменных? Как и ожидалось, например, вес на единицу площади поверхности краски $W_p$ не оказывает большого влияния на форму поверхности, тогда как коэффициент нагрузки $N_z$ (который определяет величину максимальной аэродинамической нагрузки на крыло) явно очень активен и участвует во взаимодействии с другими переменными. Классическим примером является взаимодействие с соотношением сторон $A$:  область в верхнем правом углу веса по сравнению с $A$ и $N_z$ указывает на тяжелое крыло для высоких соотношений сторон и больших g-сил (это причина, по которой высоко маневренные истребители не могут иметь очень эффективные, похожие на планеры крылья).  

Однако здесь нас интересует, насколько все это можно было бы предположить просто из незатратного скринингового исследования, без понимания инженерной значимости задействованных переменных (что довольно часто имеет место в инженерном проектировании) и без возможности вычислить такой участок плитки (что почти всегда имеет место в инженерном проектировании – в конце концов, если бы целевая функция _f_ была настолько простой для вычисления, мы бы все равно не думали о суррогатном моделировании).

Итак, что же показывает рисунок 1.2, изображающий результаты скринингового исследования при r = 5? Первое наблюдение, которое мы можем сделать, состоит в том, что существует четко определенная группа переменных, сгруппированных вокруг источника – напомним, что небольшая мера центральной тенденции(мат.ожидания???) является особенностью входных данных с небольшим влиянием на целевую функцию. Действительно, здесь мы видим, что вес краски, как и ожидалось, а также динамическое давление(скоростной напор??) имеют малую меру центральной тенденции. То же самое рассуждение применимо (и подтверждается таблицей 1.1) к коэффициенту конусности и развертке четверти хорды.  

![ssd_sm](img/ssd_sm.png)  
__Рис. 1.2.__ Оценки средних и стандартных отклонений распределений элементарных воздействий каждой из 10 переменных веса крыла в данном примере.

Хотя среднее значение все еще близко к нулю, переменная с наибольшей центральной тенденцией в этой группе - это вес топлива $W_{fw}$. Его выборка элементарных воздействий имеет очень низкое стандартное отклонение и среднее значение немного больше, чем у остальной группы, что указывает на то, что он более важен, чем предыдущие параметры, но не участвует в межпеременных взаимодействиях. График показывает, что $A$ и $tc$ имеют одинаковую важность, а из-за высоких значений стандартного отклонения они имеют нелинейный/интерактивный эффект на целевую функцию.  

Наконец, неудивительно, что большая центральная тенденция и большая мера разброса указывают на то, что $W_{dg}$, $S_w$ и $N_z$ оказывают наиболее значительное влияние на вес крыла. Конечно, авиаконструкторы знают, что общий вес самолета и площадь крыла должны быть сведены к минимуму (последнее обычно диктуется такими ограничениями, как требуемая скорость сваливания, посадочная дистанция, скорость разворота и т. д.) И что требование высокого коэффициента нагрузки $N_z$ приведет к необходимости использования прочных, тяжелых крыльев. На самом деле, именно поэтому мы использовали такую хорошо понятную функцию здесь, чтобы проиллюстрировать работу алгоритма скрининга.  

Мы кратко вернемся к вопросу установления уровня важности (или активности) переменных целевой функции в разделе о моделях Кригинга. А теперь давайте рассмотрим следующий этап процесса моделирования. С помощью активных переменных, идентифицированных (либо с помощью инженерного суждения, либо с помощью систематического скринингового исследования), мы можем теперь спроектировать основной план осуществления выборки в проектном пространстве, определяемом этими переменными. Это послужит основой для данных, на которых будет строиться суррогатная модель.

## Разработка плана по осуществлению выборки

### Стратификация

    Стратификация – это упорядочение объектов системы по уровням  (стратам) с целью создания иерархического описания системы 

Общей особенностью всех аппроксимирующих моделей, рассмотренных в этой книге, является то, что они более точны вблизи точек, в которых мы оценивали целевую функцию. В последующих главах мы познакомимся с законами, определяющими зависимость правильности модели от расстояния до этих точек. Но на данный момент мы просто сделаем интуитивный вывод: для равномерного уровня точности модели необходимо равномерное распределение точек. Такой план выборки называют _заполняющим пространство_.  
Самый простой метод такой выборки - это прямоугольная сетка точек. Это полный факториальный метод, т.е. он подвержен "проклятию размерности".  
Вот упрощённая версия функции на Python, которая должна сгенерировать полный факториальный план выборки в единичном гиперкубе.

In [37]:
def fullfactorial(q, Edges=1):
    """
    Генерирует полный факториальный план выборки в единичном кубе
    
    Вход:
        q - к-мерный вектор, содержащий число точек выборки вдоль
            каждой оси гиперкуба
        Edges - если Edges=1, то точки выборки будут браться на одинаковом
                расстоянии от края до края,
                иначе точки будут браться в центрах ячеек  n = q1*q2*...*qk,
                заполняющих единичный гиперкуб.
    Выход:
        X - полный факториальный план выборки
    """
    if np.min(q) < 2:
        print("Error: в кажом измеренеии должно быть хотя бы две точки")
        print(f"q = {q}")
        return

    #Общее число точек в плане выборки
    n = np.prod(q)
    
    #Число измерений гиперкуба
    k = len(q)
    
    #Выделим место для будущего плана выборки
    X = np.zeros((n, k))
    
    #Вспомогательный элемент
    q = np.append(q, 1)
    
    for j in np.arange(k):
        if Edges == 1:
            one_d_slice = np.hstack((np.arange(0, 1, 1 / (q[j]-1)), 1))
        else:
            one_d_slice = np.hstack((np.array(1 / (2*q[j]), 1, 1 / q[j]),1))
        
        column = np.array([])
        while len(column) < n:
            for l in np.arange(q[j]):
                column = np.concatenate([column, one_d_slice[l] * np.ones((np.prod(q[j+1:])))])
        
        X[:,j] = column
    
    return X

_Пример_ : рассмотрим обычный трёхмерный куб. Пусть (q1,q2,q3) = (3,4,5), тогда имеем следующие сетки по осям:
    1. по первой: [0, 0.5, 1]
    2. по второй: [0, 1/3, 2/3, 1]
    3. по третей: [0, 1/4, 1/2, 3/4, 1]  
    
В итоге полный факториальный план выборки $X$ имеет вид:  

In [40]:
q = np.array([3,4,5])
X = fullfactorial(q)
X

array([[0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.25      ],
       [0.        , 0.        , 0.5       ],
       [0.        , 0.        , 0.75      ],
       [0.        , 0.        , 1.        ],
       [0.        , 0.33333333, 0.        ],
       [0.        , 0.33333333, 0.25      ],
       [0.        , 0.33333333, 0.5       ],
       [0.        , 0.33333333, 0.75      ],
       [0.        , 0.33333333, 1.        ],
       [0.        , 0.66666667, 0.        ],
       [0.        , 0.66666667, 0.25      ],
       [0.        , 0.66666667, 0.5       ],
       [0.        , 0.66666667, 0.75      ],
       [0.        , 0.66666667, 1.        ],
       [0.        , 1.        , 0.        ],
       [0.        , 1.        , 0.25      ],
       [0.        , 1.        , 0.5       ],
       [0.        , 1.        , 0.75      ],
       [0.        , 1.        , 1.        ],
       [0.5       , 0.        , 0.        ],
       [0.5       , 0.        , 0.25      ],
       [0.

![fullfactorial](img/fullfactorial.bmp)  
__Рис. 1.3.__ Пример трёхмерного полнофакториального плана выборки

Очевидно, что такая конструкция удовлетворяет критерию однородности. Но она имеет два существенных недостатка.
Во-первых, такой план определён только для конструкций определённых размеров, которые могут быть записаны как произведение количеств точек вдоль каждой оси, т.е. $n = q_1 \times q_2 \times ... \times q_k$.  
Во-вторых, при проецировании на оси наборы точек будут перекрываться, и можно утверждать, что выборку для любой отдельной переменной можно улучшить, убедившись, что эти проекции являются как можно более однородными.  
Это может быть сделано путем разбиения диапазона значений этой переменной на относительно большое число ячеек одинакового размера и генерации случайных подвыборок одинакового размера внутри этих ячеек. Этот подход известен как _стратифицированная случайная выборка_. Естественным развитием этой идеи является создание плана выборки, стратифицированного по всем его измерениям. Обычно используют метод _Латинской гиперкубической выборки_.

### Латинские квадраты и случайные Латинские гиперкубы

Как уже было сказано, стратификация плана выборки направлена на создание точек, проекции которых на оси переменных однородны. Прежде чем мы рассмотрим общие методы построения таких планов, стоит рассмотреть случай дискретнозначных переменных в двух измерениях. Такие однородные проекционные планы могут быть сгенерированы довольно легко: если требуется $n$ ???точек измерения???, то квадрат $n \times n$ строится путём заполнения каждого столбца и каждой строки некоторой перестановкий $\{1,2,...,n\}$, то есть каждое число должно только один раз появляться в каждом столбце и в каждой строке. Например, для $n=4$ _Латинский квадрат_ (так обычно называют такие планы выборки) имеет вид:  


![LSQ](img/Latin_Square_n=4.bmp)  

Мы выделили единички (__1__), чтобы проиллюстрировать идею равномерной проекции, но, конечно, мы могли бы выбрать 2, 3 или 4 с тем же успехом. Кроме того, это всего лишь один произвольно выбранный четырехточечный латинский квадрат – мы могли бы с равным успехом выбрать любой из других 575 возможных вариантов. Кстати, число различных латинских квадратов довольно резко возрастает с ростом n; например, существует 108 776 032 459 082 956 800 латинских квадратов восьмого порядка! (Доказательство данного факта предоставлено читателю в качестве упражнения.)

Построение _Латинского гиперкуба_ (многомерного расширения Латинского квадрата) можно сделать аналагичным образом: разделить Область Проектирования на гиперкубы одинакового размера (ячейки) и поместить точки в эти ячейки (по одно точке в ячейку) таким образом, чтобы выполнялось следующее условие. Через каждую точку проведём линии, параллельные осям гиперкуба. Тогда каждая такая линия должна проходить ровно через одну точку. Выполнение данного условия проиллюстрировано на рис. 1.4. для трёхмерного куба  
![LHC3D](img/LatinHiperCube3D.bmp)  
__Рис. 1.4.__ Трёхпеременный десятиточечный план выборки с помощью Латинского гиперкуба. Показан в трёх измерениях(сверху слева) вместе с его двумерными проекциями. Все десять точек видны на каждой из проекций, и каждая строка и столбец ячеек содержит ровно одну точку.

Мы получаем такой план выборки следующим образом. Если $X$ - это матрица $n \times k$, в которой мы хотим записать этот план (каждая строчка обозначает одну точку) то мы начинаем заполнение $X$ случайными перестановками $\{1,2,...,n\}$ в каждом столбце и нормализуем наш план в $k$-мерном кубе $[0,1]^k$. Далее представлена реализация этого алгоритма на Python:

In [66]:
def rlh(n, k , Edges=0):
    """
    Генерирует случайный k-переменный n-точечный план выборки 
    с помощью Латинского гиперкуба
    
    Вход:
        n - желаемое количество точек
        k - число проектных переменных (количество измерений)
        Edges - если Edges=1, то крайние ячейки будут иметь свой центр 
                на краю области,
                иначе ячейки будут полностью содержаться в области
    Выход:
        X - k-переменный n-точечный План выборки с помощью 
        Латинского гиперкуба
    """
    
    #Выделим место для будущего плана выборки
    X = np.zeros((n, k))
    
    for i in np.arange(k):
        X[:,i] = 1 + np.random.permutation(n)
    if Edges==1:
        X = (X - 1) / (n-1)
    else:
        X = (X - 0.5) / n
    return X

Таким образом, приведенный выше метод даст рандомизированный план выборки, проекции которого на ось равномерно распределены (многомерная стратификация). Однако, это не гаратирует, что план будет _заполняющим пространство_ , потому что размещение всех точек на главной диагонали области проектирования будет отвечать критерию многомерной стратификации, но ,интуитивно, не будет заполнять имеющееся пространство равномерно.  
Поэтому нам нужна некоторая мера равномерности, которая позволит нам различить "хорошие" и "плохие" Латинские гиперкубы.

### Латинские гиперкубы, _заполняющие пространство_